In [10]:
import cv2
import face_recognition
import numpy as np
import os
import threading
import keyboard
import ipywidgets as widgets
from IPython.display import display, clear_output
from io import BytesIO
from PIL import Image

# Flag to indicate if 'q' has been pressed
stop_flag = False

# Function to load known faces from to_image_folder
def load_known_faces(folder):
    known_encodings = []
    known_names = []
    aadhaar_info = {}
    for filename in os.listdir(folder):
        if filename.endswith(('.jpg', '.png')):
            image_path = os.path.join(folder, filename)
            image = face_recognition.load_image_file(image_path)
            encoding = face_recognition.face_encodings(image)
            if encoding:
                known_encodings.append(encoding[0])
                name = os.path.splitext(filename)[0]  # Use filename without extension as the name
                known_names.append(name)
                aadhaar_info[name] = f"Aadhaar Details for {name}: Dummy Aadhaar {name}"
    return known_encodings, known_names, aadhaar_info

# Function to match faces
def match_faces(image_path, threshold, known_encodings, known_names):
    image = face_recognition.load_image_file(image_path)
    image_encodings = face_recognition.face_encodings(image)

    total_faces = len(image_encodings)
    matched_faces = 0

    for i, face_encoding in enumerate(image_encodings):
        matches = face_recognition.compare_faces(known_encodings, face_encoding, tolerance=threshold)
        face_distances = face_recognition.face_distance(known_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_names[best_match_index]
            matched_faces += 1
        # Calculate and display percentage completion
        percentage_complete = (i + 1) / total_faces * 100
        print(f"Matching progress: {percentage_complete:.2f}% complete", end='\r')

    print("\nMatching completed.")
    return name if matched_faces > 0 else "Unknown", None

# Function to process a selected image and match it
def process_selected_image(image_bytes, threshold):
    image = Image.open(BytesIO(image_bytes))
    image_path = os.path.join(from_image_folder, 'selected_image.jpg')
    image.save(image_path)
    
    recognized_faces.clear()
    results = []
    matched_name, matched_index = match_faces(image_path, threshold, known_face_encodings, known_face_names)
    results.append((os.path.basename(image_path), matched_name, matched_index))
    if matched_name != "Unknown":
        if matched_name in aadhaar_details and matched_name not in recognized_faces and matched_name != "Nihar":
            recognized_faces.add(matched_name)
    display_results(results)

# Function to display matching results
def display_results(results):
    result_text = ""
    for filename, name, matched_index in results:
        result_text += f"Image: {filename} - Matched Name: {name}\n"
        if matched_index is not None:
            show_matching_images(filename, matched_index)
    with output:
        clear_output()
        print(result_text)

# Function to show the two matching images
def show_matching_images(filename, matched_index):
    # Load images
    matched_image_path = os.path.join(to_image_folder, known_face_names[matched_index] + '.jpg')
    to_image_path = os.path.join(from_image_folder, filename)

    matched_image = cv2.imread(matched_image_path)
    to_image = cv2.imread(to_image_path)
    
    # Display images
    cv2.imshow('Matched From Image', matched_image)
    cv2.imshow('Matched To Image', to_image)
    cv2.waitKey(0)  # Wait for any key press
    cv2.destroyAllWindows()


# Function to update threshold and process selected image
def update_threshold(_):
    threshold = threshold_slider.value
    if uploaded_image.value:
        # Extract the uploaded file content from the tuple
        uploaded_file = uploaded_image.value[0]  # Assuming only one file is uploaded
        image_content = uploaded_file['content']
        process_selected_image(image_content, threshold)


# Function to monitor key presses
def monitor_key_presses():
    global stop_flag
    while True:
        if keyboard.is_pressed('q'):
            stop_flag = True
            break

# Load known faces
to_image_folder = 'to_image_folder'
from_image_folder = 'from_image_folder'
recognized_faces = set()  # Set to keep track of recognized faces

known_face_encodings, known_face_names, aadhaar_details = load_known_faces(to_image_folder)

# Start the key press monitoring thread
key_monitor_thread = threading.Thread(target=monitor_key_presses)
key_monitor_thread.daemon = True
key_monitor_thread.start()

# Initialize widgets
threshold_slider = widgets.FloatSlider(value=0.6, min=0.0, max=1.0, step=0.01, description='Threshold:')
uploaded_image = widgets.FileUpload(accept='.jpg,.png', multiple=False)
select_image_button = widgets.Button(description="Match Image")

# Register event listeners
select_image_button.on_click(update_threshold)

# Display widgets
display(threshold_slider)
display(uploaded_image)
display(select_image_button)

# Output widget
output = widgets.Output()
display(output)

# Function to check for the stop flag and close the OpenCV windows
def check_stop_flag():
    if stop_flag:
        cv2.destroyAllWindows()
    else:
        threading.Timer(0.1, check_stop_flag).start()

# Start checking for the stop flag
check_stop_flag()


FloatSlider(value=0.6, description='Threshold:', max=1.0, step=0.01)

FileUpload(value=(), accept='.jpg,.png', description='Upload')

Button(description='Match Image', style=ButtonStyle())

Output()